In [1]:
import re, json, time, requests, sys, os
from typing import List, Dict, Any, Optional
from bs4 import BeautifulSoup
from requests.adapters import HTTPAdapter
from urllib3.util.retry import Retry
from dotenv import load_dotenv

# 환경변수 로드
load_dotenv()

# KB 차차차 URL
KB_HOST = "https://www.kbchachacha.com"
DETAIL_URL = f"{KB_HOST}/public/car/detail.kbc"
INSURANCE_HISTORY_INFO_URL = f"{KB_HOST}/public/car/layer/member/car/history/info.kbc"

# 세션 생성
session = requests.Session()
session.headers.update({
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/140.0.0.0 Safari/537.36"
})

In [2]:
# 테스트 차량
car_seq = "25574426"

print(f"테스트 차량: carSeq={car_seq}")
print("=" * 80)


테스트 차량: carSeq=25574426


In [3]:
# 1단계: 차량 상세 페이지 접속 (로그인 없이)
print("\n[1단계] 차량 상세 페이지 접속 (로그인 없이)...")
detail_response = session.get(f"{DETAIL_URL}?carSeq={car_seq}", timeout=15)

print(f"  - 상태 코드: {detail_response.status_code}")
print(f"  - 응답 길이: {len(detail_response.text):,}자")

# carHistorySeq 추출 시도
detail_soup = BeautifulSoup(detail_response.text, 'html.parser')
car_history_seq = None

scripts = detail_soup.find_all('script')
for script in scripts:
    if script.string and 'carHistorySeq' in script.string:
        match = re.search(r'carHistorySeq["\s:=]+(\d+)', script.string)
        if match:
            car_history_seq = match.group(1)
            print(f"  - carHistorySeq 발견: {car_history_seq}")
            break

if not car_history_seq:
    print("  - carHistorySeq를 찾을 수 없음")



[1단계] 차량 상세 페이지 접속 (로그인 없이)...
  - 상태 코드: 200
  - 응답 길이: 213,328자
  - carHistorySeq 발견: 44288099


In [4]:
# 2단계: 보험이력 요청 (로그인 없이)
print("\n[2단계] 보험이력 요청 보내기 (로그인 없이)...")

post_data = {
    "layerId": "layerCarHistoryInfo",
    "refGbn": "331100",
    "refSeq": car_seq
}

if car_history_seq:
    post_data["carHistorySeq"] = car_history_seq

print(f"  - POST 데이터: {post_data}")

insurance_response = session.post(
    INSURANCE_HISTORY_INFO_URL,
    data=post_data,
    headers={
        "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8",
        "Content-Type": "application/x-www-form-urlencoded",
        "Referer": f"{DETAIL_URL}?carSeq={car_seq}",
        "Origin": "https://www.kbchachacha.com"
    },
    timeout=15
)

print(f"  - 상태 코드: {insurance_response.status_code}")
print(f"  - 응답 길이: {len(insurance_response.text):,}자")



[2단계] 보험이력 요청 보내기 (로그인 없이)...
  - POST 데이터: {'layerId': 'layerCarHistoryInfo', 'refGbn': '331100', 'refSeq': '25574426', 'carHistorySeq': '44288099'}
  - 상태 코드: 500
  - 응답 길이: 36,202자


In [5]:
# 3단계: 응답 분석
print("\n[3단계] 응답 분석...")

soup = BeautifulSoup(insurance_response.text, 'html.parser')

# "등록된 사고이력조회 정보가 없습니다" 메시지 확인
no_data_div = soup.find('div', class_='ch-none')
if no_data_div:
    print("  ❌ 로그인 필요: '등록된 사고이력조회 정보가 없습니다' 메시지 발견")
else:
    print("  ✅ 데이터 있음!")
    
# acd-qk-list 확인
acd_lists = soup.find_all('div', class_='acd-qk-list')
print(f"  - acd-qk-list 개수: {len(acd_lists)}")

# 내차 피해 정보 확인
for acd_list in acd_lists:
    items = acd_list.find_all('li')
    for item in items:
        tit = item.find('span', class_='tit')
        txt = item.find('span', class_='txt')
        if tit and txt:
            tit_text = tit.get_text(strip=True)
            txt_text = txt.get_text(strip=True)
            if '내차 피해' in tit_text or '상대차 피해' in tit_text:
                print(f"  - {tit_text}: {txt_text}")



[3단계] 응답 분석...
  ✅ 데이터 있음!
  - acd-qk-list 개수: 0


In [7]:
# 4단계: HTML 일부 출력 (디버깅용)
print("\n[4단계] HTML 응답 일부 출력...")
print("="*80)
print(insurance_response.text)
print("="*80)



[4단계] HTML 응답 일부 출력...










    
    	
    
    

<!DOCTYPE html>
<html lang="ko">
<head>
<meta charset="utf-8">
<meta http-equiv="X-UA-Compatible" content="IE=edge">







<meta name="facebook-domain-verification" content="qu29xyvgvk5x9719cld2jdgmmqndt4" />

  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
    <title>KB차차차 - 국내 최다 중고차 매물 등록대수 1위!</title>
    <meta name="description"
          content="중고차 매매 사이트, 내차팔기, 중고차 경매, 자동차 시세, 중고차 판매, 수입차, 자동차 리스, 자동차 렌트, 인증중고차, 안심 중고차 플랫폼 KB차차차 공식 홈페이지">

    <meta property="og:title" content="KB차차차 - 국내 최다 중고차 매물 등록대수 1위!" />
    <meta property="og:description"
          content="중고차 매매 사이트, 내차팔기, 중고차 경매, 자동차 시세, 중고차 판매, 수입차, 자동차 리스, 자동차 렌트, 인증중고차, 안심 중고차 플랫폼 KB차차차 공식 홈페이지" />
    <meta property="og:image" content="https://www.kbchachacha.com/images/common/og-thumb.png" /> 
  


<meta pro